In [1]:
import zarr
import numpy as np
import pystac_client as pc
import xarray as xr
import rioxarray
import pandas as pd
from datetime import datetime, timezone, date
#from tqdm import tqdm
import BuildZarrStore as bzs
import stackstac
from pyproj import CRS
from dask.distributed import LocalCluster
#os.environ["ZARR_V3_EXPERIMENTAL_API"] = "1"

In [ ]:
origin = np.datetime64("2014-10-01")
date = np.datetime64(datetime.now()).astype("datetime64[D]")
date_delta = (date - origin+1).astype("int64")

date_delta

In [ ]:
x_extent = np.arange(4800000, 5700000, 20)
y_extent = np.arange(1800000, 1200000, -20)
time_extent = np.arange(0,date_delta,1)

In [ ]:
shape = (time_extent.shape[0],y_extent.shape[0],x_extent.shape[0])
chunk_shape = (30,500,500)
shard_shape = (30,2500,2500)
x_shape = x_extent.shape
y_shape = y_extent.shape
time_shape = time_extent.shape

In [ ]:
overwrite=True
store = zarr.storage.LocalStore("s1sig0_v2.zarr")
root = zarr.create_group(store=store, overwrite=overwrite)
vv = root.create_group("VV")
vh = root.create_group("VH")
meta = root.create_group("metadata")

In [ ]:
bzs.make_group_S1(vv, "VV", shape, shards=shard_shape, chunks=chunk_shape, x_shape=x_shape, y_shape=y_shape, time_shape=time_shape, x_extent=x_extent, y_extent=y_extent, time_extent=time_extent)
bzs.make_group_S1(vh, "VH", shape, shards=shard_shape, chunks=chunk_shape, x_shape=x_shape, y_shape=y_shape, time_shape=time_shape, x_extent=x_extent, y_extent=y_extent, time_extent=time_extent)

In [ ]:
meta.create_array(name="sensing_date",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int64",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"units": "seconds since 2014-10-01 00:00:00",
                            "calendar": "proleptic_gregorian",
                            "_FillValue": -9999},
                overwrite=overwrite)

meta.create_array(name="absolute_orbit_number",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int32",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"_FillValue": -9999},
                overwrite=overwrite)

meta.create_array(name="relative_orbit_number",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"_FillValue": -9999},
                overwrite=overwrite)

x_array = meta.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8="}, #fill value is NaN
            overwrite=overwrite)

y_array = meta.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8="}, #fill value is NaN
            overwrite=overwrite)

time_array = meta.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2014-10-01",
                        "calendar": "proleptic_gregorian"},
            overwrite=overwrite)

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

In [ ]:
zarr.consolidate_metadata(store, path="VV")
zarr.consolidate_metadata(store, path="VH")
zarr.consolidate_metadata(store, path="metadata")
zarr.consolidate_metadata(store)

In [ ]:
store = zarr.storage.LocalStore("s1sig0_v2.zarr")
group = zarr.group(store=store)["VV"]
x_extent = group["x"][:]
y_extent = group["y"][:]
time_shape = group["time"].shape
data_shape = group["VV"].shape

In [ ]:
time2append = np.arange(group["time"][-1]+1, group["time"][-1]+31, 1)

In [ ]:
group["VV"].resize((3962, 30000, 45000))

In [ ]:
group["time"].append(time2append)

In [ ]:
with open('s1sig0_v2.zarr/zarr.json', 'w') as f:
    f.write("{}")

In [ ]:
zarr.consolidate_metadata(store, path="VV")
zarr.consolidate_metadata(store, path="VH")
zarr.consolidate_metadata(store, path="metadata")
zarr.consolidate_metadata(store)

In [5]:
xr.open_zarr("INCA_old.zarr", group="GL",consolidated = True)

<xarray.Dataset> Size: 282GB
Dimensions:  (time: 125328, y: 401, x: 701)
Coordinates:
  * time     (time) datetime64[ns] 1MB 2011-03-15 2011-03-15 ... 2011-03-15
  * y        (y) float64 3kB 2.2e+05 2.21e+05 2.22e+05 ... 6.19e+05 6.2e+05
  * x        (x) float64 6kB 2e+04 2.1e+04 2.2e+04 ... 7.18e+05 7.19e+05 7.2e+05
Data variables:
    GL       (time, y, x) float64 282GB dask.array<chunksize=(720, 100, 100), meta=np.ndarray>
Attributes: (12/13)
    Description:         global radiation
    Unit:                W m-2
    name:                incal-hourly
    Conventions:         CF-1.7
    freq:                1H
    spatial_resolution:  1000
    ...                  ...
    Institution:         GeoSphere Austria
    history:             Original data produced by GeoSphere Austria
    source:              modeled data
    crs:                 EPSG: 31287
    grid_mapping:        lambert_conformal_conic
    Version:             1